In [1]:
from itertools import chain
import json
import os
from pathlib import Path
import sys

from arcgis.features import GeoAccessor
from arcgis.geometry import Geometry
import pandas as pd

In [2]:
project_parent = Path('./').absolute().parent

data_dir = project_parent/'data'

data_raw = data_dir/'raw'
data_int = data_dir/'interim'
data_out = data_dir/'processed'

gdb_raw = data_raw/'raw.gdb'
gdb_int = data_int/'interim.gdb'
gdb_out = data_out/'processed.gdb'

# import the project package from the project package path
sys.path.append(str(project_parent/'src'))
import h3_arcgis

# load the "autoreload" extension so that code can change, & always reload modules so that as you change code in src, it gets loaded
%load_ext autoreload
%autoreload 2

# Start Analysis Prototyping

In [3]:
raw_trips_csv = data_raw/'raw_trips.csv'
raw_df = pd.read_csv(raw_trips_csv, index_col=0).reset_index(drop=True)
raw_df.SHAPE = raw_df.SHAPE.swifter.apply(lambda geom: Geometry(eval(geom)))
raw_df.spatial.set_geometry('SHAPE')
raw_df.head()

C:\Program Files\ArcGIS\Pro\bin\Python\envs\h3-py\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,travel_distance_miles,travel_time_minutes,SHAPE,store_locnum
0,13.261921,25.033333,"{'x': -123.026415, 'y': 44.87728, 'spatialRefe...",413963145
1,8.260506,12.466667,"{'x': -123.150157, 'y': 44.933511, 'spatialRef...",244210043
2,2.013863,8.750000,"{'x': -123.312642, 'y': 44.929472, 'spatialRef...",244210043
3,0.547428,7.516667,"{'x': -123.30801, 'y': 44.9299, 'spatialRefere...",244210043
4,0.133595,3.100000,"{'x': -123.315767, 'y': 44.922983, 'spatialRef...",244210043


In [4]:
fnl_df = h3_arcgis.get_nonoverlapping_h3_hexbins_for_points(raw_df, h3_max=10)
fnl_df.head()

,h3_id,SHAPE,count,h3_lvl
0,8628f001fffffff,"{'rings': [[[-122.63472201299999, 45.569528682...",16690,6.0
1,8728f0129ffffff,"{'rings': [[[-122.75436712599998, 45.382967921...",14566,7.0
2,8628f00a7ffffff,"{'rings': [[[-122.49997860599996, 45.550465284...",13217,6.0
3,8628f00e7ffffff,"{'rings': [[[-122.69069125499999, 45.605834104...",12610,6.0
4,8628f42efffffff,"{'rings': [[[-123.06206813099999, 44.887612749...",10828,6.0


In [5]:
fnl_df.spatial.plot()

MapView(layout=Layout(height='400px', width='100%'))